In [13]:
import tensorflow as tf

In [14]:
!wget --no-check-certificate \
  ftp://cs.stanford.edu/cs/cvgl/Stanford_Online_Products.zip \
  -O /tmp/Stanford_Online_Products.zip

--2021-09-29 07:44:04--  ftp://cs.stanford.edu/cs/cvgl/Stanford_Online_Products.zip
           => ‘/tmp/Stanford_Online_Products.zip’
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /cs/cvgl ... done.
==> SIZE Stanford_Online_Products.zip ... 3083860082
==> PASV ... done.    ==> RETR Stanford_Online_Products.zip ... done.
Length: 3083860082 (2.9G) (unauthoritative)

Stanford_Online_Pro 100%[===================>]   2.87G  18.0MB/s    in 2m 50s  

2021-09-29 07:46:56 (17.3 MB/s) - ‘/tmp/Stanford_Online_Products.zip’ saved [3083860082]



In [15]:
import zipfile, os

file_zip = '/tmp/Stanford_Online_Products.zip'
zip_ref = zipfile.ZipFile(file_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

base_dir = '/content/Stanford_Online_Products'

In [17]:
os.listdir(base_dir)

['table_final.txt',
 'sofa_final',
 'coffee_maker_final',
 'cabinet_final',
 'sofa_final.txt',
 'chair_final',
 'fan_final.txt',
 'stapler_final',
 'Ebay_test.txt',
 'chair_final.txt',
 'kettle_final',
 'LICENSE',
 'bicycle_final',
 'fan_final',
 'coffee_maker_final.txt',
 'lamp_final',
 'table_final',
 'stapler_final.txt',
 'mug_final.txt',
 'toaster_final',
 'toaster_final.txt',
 'README',
 'cabinet_final.txt',
 'Ebay_info.txt',
 'kettle_final.txt',
 'lamp_final.txt',
 'Ebay_final.txt',
 'mug_final',
 'bicycle_final.txt',
 'Ebay_train.txt']

In [18]:
# proses augmentasi gambar
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
                    rescale = 1./225,
                    rotation_range = 20,
                    horizontal_flip = True,
                    shear_range = 0.2,
                    fill_mode = 'nearest',
                    validation_split = 0.2
)

In [21]:
# deklarasi variabel global ukuran gambar
image_size = (150, 150)

# mempersiapkan data latih dan data validasi yang akan dipelajari oleh model
train_generator = train_datagen.flow_from_directory(
        base_dir,  # direktori data latih
        target_size = image_size,  # mengubah resolusi seluruh gambar menjadi 150x150 piksel dengan variabel global
        classes=['lamp_final', 'mug_final', 'sofa_final'],
        batch_size = 128,
        class_mode = 'categorical', # karena kita merupakan masalah klasifikasi 3 kelas maka menggunakan class_mode = 'categorical'
        subset='training'
)
 
val_generator = train_datagen.flow_from_directory(
        base_dir, # direktori data validasi
        target_size = image_size, # mengubah resolusi seluruh gambar menjadi 150x150 piksel dengan variabel global
        classes=['lamp_final', 'mug_final', 'sofa_final'],
        batch_size = 64,
        class_mode = 'categorical', # karena kita merupakan masalah klasifikasi 3 kelas maka menggunakan class_mode = 'categorical'
        subset='validation'
)

Found 26870 images belonging to 3 classes.
Found 6717 images belonging to 3 classes.


In [22]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (150,150,3)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2))) 
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Conv2D(filters =96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Conv2D(filters = 96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(3, activation = "softmax")) # untuk dataset dengan 3 kelas atau lebih menggunakan fungsi aktivasi softmax

In [23]:
# compile model dengan 'adam' optimizer loss function 'categorical_crossentropy'
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = tf.optimizers.Adam(),
    metrics = ['accuracy']
)

In [27]:
# deklarasi variabel global epoch
epochs = 50
# membuat objek callback untuk early stopping dan reduksi learning rate
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience=3, verbose=2, factor=0.1)

# latih model dengan model.fit
model.fit(
    train_generator,
    epochs = epochs, 
    validation_data = val_generator,
    verbose = 2, 
    steps_per_epoch = train_generator.samples // train_generator.batch_size - 1,
    callbacks=[reduce_lr],
    validation_steps = val_generator.samples // val_generator.batch_size - 1
)


Epoch 1/50


KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy Model')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss Model')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()